# Aesop's Fables
AI to learn Aesop's Fables and create more stories

![Aesop's Fables](https://images-na.ssl-images-amazon.com/images/I/51DM5uYLhZL._SX258_BO1,204,203,200_.jpg)


The model is built using a neural network with a LSTM (long short term memory) layer. The data are text from Aesop's fables. It's a relatively small dataset of only 188540 characters.

**Live demo: https://laesop-fables-generator.glitch.me/**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# get the data
data = __import__('requests').get('https://raw.githubusercontent.com/aunz/ds/master/.data/Aesops_Fables.txt').text[:-50]
print('Corpus length', len(data))

Corpus length 188540


In [5]:
# extract the data into sentences and next_chars
sentences = [] # store the extracted sequences
next_chars = [] # store the target (the follow-up characters)
max_len = 60 # extract sequences of 60 characters
step = 2 # sample a new sequence every two characters

for i in range(0, len(data) - max_len, step):
    sentences.append(data[i: i + max_len])
    next_chars.append(data[i + max_len])

print('Number of sequences:', len(sentences))

Number of sequences: 94240


In [6]:
chars = sorted(list(set(data))) # list of unique char in the corpus
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) # turn the list into dict to map unique chars to their index in the list "chars"
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences): # one hot encode into binary arrays
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

Unique characters: 64
Vectorization...


In [7]:
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense

# a simple model of LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.01))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
Total params: 107,072
Trainable params: 107,072
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds, temperature=0.5):
    ''' function to sample the next char given the model's prediction
    Because given the same input, the predicted output will always be the same.
    This function take an array of preds (sum to 1), and a temperature param, return a new array of preds, then randomly select 1 element
    At a low temp, the new array of preds has a distribution similar to the old array
    Conversely, at a high temp, the new array of preds will be quite different from the old array
    Based on the new distribution of pred probability, the numpy np.random.multinomial is used to randomly select 1 element
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log1p(preds) / temperature
    preds = np.expm1(preds)
    preds = preds / np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def predict(text_seed, n_char=420, temperature=0.5):
    # given the seed_text, predict up to the next n_char
    generated_text = f'{text_seed:>60}'[:max_len] # pad string to the right and only take up to max_len
    for i in range(n_char):
        sampled = np.zeros((1, max_len, len(chars))) # one hot encodes the chars generated so far
        for t, char in enumerate(generated_text):
            if char in char_indices: sampled[0, t, char_indices[char]] = 1.
        preds = model.predict(sampled, verbose=0)[0] # sample the next char
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        text_seed += next_char
    return text_seed

In [0]:
%%time
import time

start_time = time.time()
for epoch in range(1, 60): # train the model for 60 epochs
    start_time_inner = time.time()
    print('Epoch', epoch, end=' ')
    model.fit(x, y, batch_size=128, epochs=1) # fit the model for 1 iteration on the data
    start_index = np.random.randint(0, len(data) - max_len - 1) # select a text seed at random
    text_seed = data[start_index: start_index + max_len]
#     text_seed = 'A Bat who fell upon the ground and was '
    print('--- Generating with seed: "' + text_seed + '"')
    for temperature in [0.2, 0.5, 0.9]:
        print('------ temp:', temperature, end='. ')
        predicted_text = predict(text_seed, temperature=temperature)
        print(predicted_text)
    print('Elapsed. This loop:', time.time() - start_time_inner, '. Total:', time.time() - start_time, '\n')

# model = load_model('./model_60.h5') # or load a saved model

In [0]:
model.save('./model_60.h5')
# convert to .bin for browser
!curl https://raw.githubusercontent.com/transcranial/keras-js/master/python/encoder.py -o encoder.py
!curl https://raw.githubusercontent.com/transcranial/keras-js/master/python/model_pb2.py -o model_pb2.py
!python encoder.py -q ./model_60.h5

In [0]:
predict('The Hare and the Tortoise')

**Credits:**
- Deep Learning with Python by [Francois Chollet](https://github.com/fchollet)
- http://classics.mit.edu/Aesop/fab.mb.txt
